In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
# Loads the aligned trajectory
import MDAnalysis as mda
ref = mda.Universe('../sarscov2-10921231-structure/protein_conf.gro')
sim = mda.Universe('../sarscov2-10921231-structure/protein_conf.gro', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_00to02us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_02to04us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_04to06us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_06to08us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_08to10us.xtc')

# Principal components analysis

In [ ]:
import MDAnalysis.analysis.pca as pca
sim_pca = pca.PCA(sim, select='protein and name CA')
sim_pca.run(start=0)

## Visualize the principal components

In [ ]:
X = sim_pca.mean[::3]
Y = sim_pca.mean[1::3]
Z = sim_pca.mean[2::3]

for n in range(5):
  U = sim_pca.p_components[::3,n]
  V = sim_pca.p_components[1::3,n]
  W = sim_pca.p_components[2::3,n]

  from mpl_toolkits.mplot3d import Axes3D
  fig = plt.figure(n+1)
  ax = fig.add_subplot(111, projection='3d')

  lengths = [np.sqrt(u**2 + v**2 + w**2) for (u, v, w) in zip(U, V, W)]
  # ax.quiver(X, Y, Z, U, V, W, length=10)
  for (x, y, z, u, v, w) in zip(X, Y, Z, U, V, W):
    length = np.sqrt(u**2 + v**2 + w**2)
    ax.quiver(x, y, z, u, v, w, length=100*length/np.max(lengths), pivot='tail')

In [ ]:
X = sim_pca.mean[::3]
Y = sim_pca.mean[1::3]
Z = sim_pca.mean[2::3]

resid = list(np.repeat(range(1,307),3))
for n in range(3):
  plt.figure(n)
  plt.plot(resid, sim_pca.p_components[:306*3,n], '.')
  plt.plot(resid, sim_pca.p_components[306*3:,n], '.')
  plt.legend(['subunit 1','subunit 2'])
  plt.xlabel('Residue')
  plt.ylabel('Weight')

In [ ]:
import numpy as np
print('The dot product of the first PC with itself is', np.dot(sim_pca.p_components[:,0],sim_pca.p_components[:,0]))
print('The dot product of the first and second PCs is', np.dot(sim_pca.p_components[:,0],sim_pca.p_components[:,1]))

## The variance and cumulated variance

In [ ]:
plt.plot(sim_pca.variance,'.-')
plt.xlabel('Principal component')
plt.ylabel('Eigenvalue')

In [ ]:
plt.subplot(2,1,1)
plt.plot(sim_pca.cumulated_variance[:10],'.-')
plt.xlabel('Principal component');
plt.ylabel('Cumulative variance');
plt.subplot(2,1,2)
plt.plot(sim_pca.cumulated_variance,'.-')
plt.xlabel('Principal component');
plt.ylabel('Cumulative variance');

## Projections onto the most important principal components

In [ ]:
atomgroup = sim.select_atoms('protein and name CA')
pca_space = sim_pca.transform(atomgroup, n_components=10)

In [ ]:
for n in range(5):
  plt.figure(n)
  plt.plot(pca_space[:,n],'.-')
  plt.xlabel('Frame')
  plt.ylabel(f'Projection onto PC {n+1}')

In [ ]:
plt.scatter(pca_space[:,0], pca_space[:,1], c=range(pca_space.shape[0]), cmap="RdYlGn")
plt.colorbar()